In [1]:
import numpy as np
from math import factorial as fact
from functools import lru_cache
from itertools import product

from utils import *

In [2]:
@lru_cache(maxsize=100000)
def Lambda(l,a,b):
    a_arr, b_arr = np.array(a),np.array(b)
    
    if card(a_arr)==0 or card(b_arr)==0 or l<2:
        return fact(l-1)
    
    
    if norm(a_arr)+norm(b_arr)==l-1:
        if magn(a_arr)<l and magn(b_arr)<=l:
            answer = 0
            for i in range(2,min(l+1,len(b)+2)):
                if b[i-2] > 0:
                    answer += (i-1)*partialterm(l,i,b_arr)*Lambda(l-1,a,tuple(partial(i,b)))
            return l*answer // (l-magn(a_arr))

        elif magn(a_arr)<=l and magn(b_arr)<l:
            answer = 0
            for i in range(2,min(l+1,len(a)+2)):
                if a[i-2] > 0:
                    answer += l*(i-1)*partialterm(l,i,a_arr)*Lambda(l-1,b,tuple(partial(i,a)))
            return answer//(l-magn(b_arr))
    return 0

In [3]:
cycle_coeff = lambda l,a,b: Lambda(l,a,b)//fact(l-1)

In [4]:
## lambda(9,[2,..],[1,0,0,0,1,...]) = 54
cycle_coeff(9,
            (2,0,0,0,0,0),
            (1,0,0,0,1,0))

54

In [5]:
Lambda.cache_info()

CacheInfo(hits=4, misses=12, maxsize=100000, currsize=12)

In [6]:
def gen_cdecomp(a,c):
    a_arr = np.array(a)
    nu = np.nonzero(c)[0][0] +2
    
    intervals = [range(0,min(a[i],nu)+1) for i in range(0,min(nu,len(a)))]
    
    cart_prod = product(*intervals)
    
    answer = []
    for A in cart_prod:
        A_arr = np.array(A)
        if magn(A_arr)<=nu:
            
            answer.append((A,tuple(a_arr[0:len(A_arr)]-A_arr)))
        
    return answer

In [7]:
gen_cdecomp((0,1,2,3),(0,0,2,1))

[((0, 0, 0, 0), (0, 1, 2, 3)),
 ((0, 0, 1, 0), (0, 1, 1, 3)),
 ((0, 1, 0, 0), (0, 0, 2, 3))]

In [8]:
def Lambda2(c,a,b):
    if norm(a) + norm(b) != norm(c):
        return 0
    
    if sum(a)==0 and sum(b)==0 and sum(c) == 0:
        return 1

    nu = np.nonzero(c)[0][0] +2
    a_decomps = gen_cdecomp(a,c)
    b_decomps = gen_cdecomp(b,c)
    
    answer = 0
    for A1,A2 in a_decomps:
        for B1,B2 in b_decomps:
            if norm(np.array(A1))+norm(np.array(B1)) == nu-1:
                answer += cycle_coeff(nu,A1,B1) * Lambda2(c - np.eye(1,len(c),nu-2,dtype=int)[0], A2,B2)
    return answer

In [9]:
Lambda2((1,1,1),(3,0,0,0,0,0,0),(3,0,0,0,0,0))

12

## Generate tables

In [10]:
def accel_asc(n):
    a = [0 for i in range(n + 1)]
    k = 1
    y = n - 1
    while k != 0:
        x = a[k - 1] + 1
        k -= 1
        while 2 * x <= y:
            a[k] = x
            y -= x
            k += 1
        l = k + 1
        while x <= y:
            a[k] = x
            a[l] = y
            yield a[:k + 2]
            x += 1
            y -= 1
        a[k] = x + y
        y = x + y - 1
        yield a[:k + 1]

In [11]:
def to_ctype(l):
    answer = np.zeros(max(l),dtype=int)
    for i in l:
        answer[i-1] += 1
    return tuple(answer)

In [14]:

a,b = (5,),(5,)

print("Multiplication of: ", a, "and ", b)


for i in accel_asc(10):
    print("value of c:", to_ctype(i), "\n", "coeff", Lambda2(to_ctype(i),a,b))

Multiplication of:  (5,) and  (5,)
value of c: (10,) 
 coeff 252
value of c: (8, 1) 
 coeff 210
value of c: (7, 0, 1) 
 coeff 140
value of c: (6, 2) 
 coeff 180
value of c: (6, 0, 0, 1) 
 coeff 100
value of c: (5, 1, 1) 
 coeff 120
value of c: (5, 0, 0, 0, 1) 
 coeff 60
value of c: (4, 3) 
 coeff 162
value of c: (4, 1, 0, 1) 
 coeff 90
value of c: (4, 0, 2) 
 coeff 80
value of c: (4, 0, 0, 0, 0, 1) 
 coeff 42
value of c: (3, 2, 1) 
 coeff 108
value of c: (3, 1, 0, 0, 1) 
 coeff 54
value of c: (3, 0, 1, 1) 
 coeff 60
value of c: (3, 0, 0, 0, 0, 0, 1) 
 coeff 24
value of c: (2, 4) 
 coeff 162
value of c: (2, 2, 0, 1) 
 coeff 90
value of c: (2, 1, 2) 
 coeff 72
value of c: (2, 1, 0, 0, 0, 1) 
 coeff 42
value of c: (2, 0, 1, 0, 1) 
 coeff 36
value of c: (2, 0, 0, 2) 
 coeff 50
value of c: (2, 0, 0, 0, 0, 0, 0, 1) 
 coeff 18
value of c: (1, 3, 1) 
 coeff 108
value of c: (1, 2, 0, 0, 1) 
 coeff 54
value of c: (1, 1, 1, 1) 
 coeff 60
value of c: (1, 1, 0, 0, 0, 0, 1) 
 coeff 24
value of c: (1